Project 1 Solutions - California Water Usage
=================

Welcome to the first project in DSC 10! We will be exploring possible connections between water usage, geography, and income in California. The water data for this project was procured from the [California State Water Resources Control Board](http://www2.pacinst.org/gpcd/table.html) and curated by the [Pacific Institute](http://pacinst.org/). The map data includes [US topography](https://github.com/jgoodall/us-maps), [California counties](https://github.com/johan/world.geo.json/tree/master/countries/USA/CA), and [ZIP codes](http://bl.ocks.org/jefffriesen/6892860).

The dataset on income comes from the IRS ([documentation](http://www.irs.gov/pub/irs-soi/13zpdoc.doc)).  We have identified some interesting columns in the dataset, but a full description of all the columns (and a definition of the population in the dataset and some interesting anonymization procedures they used) is available in this [description](irs_info.pdf).

As usual, **run the cell below** to prepare the automatic tests. **Passing the automatic tests does not guarantee full credit on any question.** The tests are provided to help catch some common errors, but it is *your* responsibility to answer the questions correctly.

In [1]:
# Run this cell, but please don't change it.

import numpy as np
import math
from datascience import *

# These lines set up the plotting functionality and formatting.
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# These lines load the tests.
from client.api.assignment import load_assignment 
from client.api.notebook import Notebook
project1 = Notebook('project1.ok')
_ = project1.auth(inline=True)

Assignment: Project 1
OK, version v1.12.8


Open the following URL:

https://okpy.org/client/login/

After logging in, copy the code from the web page and paste it into the box.
Then press the "Enter" key on your keyboard.

Paste your code here: Ejcz7zT4D8Cbv5GMZmSRYAKITC6nIH
Successfully logged in as jlobue@eng.ucsd.edu


First, load the data. Loading may take some time.

In [2]:
# Run this cell, but please don't change it.

districts = Map.read_geojson('water_districts.geojson')
zips = Map.read_geojson('ca_zips.geojson.gz')
usage_raw = Table.read_table('water_usage.csv', dtype={'pwsid': str})
income_raw = Table.read_table('ca_income_by_zip.csv', dtype={'ZIP': str}).drop(['STATEFIPS', 'STATE', 'agi_stub'])
wd_vs_zip = Table.read_table('wd_vs_zip.csv', dtype={'PWSID': str, 'ZIP': str}).set_format([2, 3], PercentFormatter)

Part 0: Maps
======

The `districts` and `zips` data sets are `Map` objects. Documentation on mapping in the `datascience` package can be found at [here](http://data8.org/datascience/maps.html).

###### To view a map of the water districts in California, please click [here](https://glowingshadow.ml:8888/notebooks/dsc10-fa17/projects/proj1/project1.ipynb) (Password is DSC10). Please be patient, this takes some time to load.

A `Map` is a collection of regions and other features such as points and markers, each of which has a **string** `id` and various properties. You can view the features of the `districts` map as a table using `Table.from_records`.

In [3]:
district_table = Table.from_records(districts.features)
district_table.show(3)

PWSID,feature,id,popupContent
0110001,""" style=""width: 180px; height: 180px""; border: none"">",0,Alameda County Water District
0110003,""" style=""width: 180px; height: 180px""; border: none"">",1,California Water Service Company Livermore
0110005,""" style=""width: 180px; height: 180px""; border: none"">",2,East Bay Municipal Utilities District


To display a `Map` containing only two features from the `district_table`, call `Map` on a list containing those two features from the `feature` column.
    
**Question 0.0** Draw a map of the Alameda County Water District (row 0) and the East Bay Municipal Utilities District (row 2).

In [4]:
alameda_and_east_bay = [district_table.column(1).item(0), district_table.column(1).item(2) ]
Map(alameda_and_east_bay, height=300, width=300)

In [5]:
_ = project1.grade('q00')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



*Hint*: If scrolling becomes slow on your computer, you can clear maps for the cells above by running `Cell > All Output > Clear` from the `Cell` menu. 

Part 1: California Income
=======

Let's look at the `income_raw` table.

In [6]:
income_raw

ZIP,N1,MARS1,MARS2,MARS4,PREP,N2,NUMDEP,A00100,N02650,A02650,N00200,A00200,N00300,A00300,N00600,A00600,N00650,A00650,N00700,A00700,N00900,A00900,N01000,A01000,N01400,A01400,N01700,A01700,SCHF,N02300,A02300,N02500,A02500,N26270,A26270,N02900,A02900,N03220,A03220,N03300,A03300,N03270,A03270,N03150,A03150,N03210,A03210,N03230,A03230,N03240,A03240,N04470,A04470,A00101,N18425,A18425,N18450,A18450,N18500,A18500,N18300,A18300,N19300,A19300,N19700,A19700,N04800,A04800,N05800,A05800,N09600,A09600,N07100,A07100,N07300,A07300,N07180,A07180,N07230,A07230,N07240,A07240,N07220,A07220,N07260,A07260,N09400,A09400,N10600,A10600,N59660,A59660,N59720,A59720,N11070,A11070,N10960,A10960,N06500,A06500,N10300,A10300,N85330,A85330,N85300,A85300,N11901,A11901,N11902,A11902
90001,13100,6900,1890,4270,10740,29670,15200,181693,13100,184344,10220,142287,640,96,50,69,30,20,180,55,3490,31751,70,-17,60,382,380,3001,0,950,4493,80,123,30,105,3510,2651,20,6,0,0,0,0,0,0,150,119,60,98,0,0,510,7212,8498,170,95,270,154,340,952,500,1424,340,3168,240,363,4200,20798,4190,2176,0,0,1800,623,0,0,70,28,520,248,240,39,1020,304,0,0,3290,4659,12180,37759,6320,20531,5880,16995,6500,8805,1230,980,2680,1554,5800,6197,0,0,0,0,1220,856,11480,32934
90001,5900,1700,1970,2210,4960,17550,9690,203628,5900,204512,5610,188556,830,68,60,32,70,19,660,351,540,5917,60,111,50,489,310,4450,0,460,2522,210,947,40,831,700,885,50,12,0,0,0,0,50,152,200,154,50,81,0,0,1260,20442,46800,800,1023,430,505,810,2383,1250,4605,820,7172,870,1620,5300,74352,5290,8710,0,0,3380,2947,0,0,370,249,620,518,630,114,2520,2047,20,6,410,982,5820,23531,2160,4562,1890,3948,2520,4160,600,441,3180,5762,3460,6754,0,0,0,0,530,797,5360,17656
90001,1480,330,760,390,1240,4710,2470,89065,1480,89344,1440,82579,400,32,20,12,0,0,460,477,130,1351,40,1,40,660,140,2599,0,150,892,60,670,0,0,210,279,50,14,0,0,0,0,0,0,90,76,0,0,0,0,680,13264,41920,540,1329,120,294,500,1453,680,3591,490,4248,560,1296,1470,49298,1470,6618,0,0,900,1488,0,0,120,90,180,198,80,11,750,1163,20,7,80,232,1470,8950,0,0,0,0,190,256,170,132,1290,5131,1310,5417,0,0,0,0,230,520,1250,4050
90001,330,50,210,70,290,1100,560,28395,330,28555,320,25271,130,10,20,24,30,29,160,235,60,1084,0,0,0,0,40,1044,0,40,230,30,598,0,0,60,160,0,0,0,0,0,0,0,0,30,21,0,0,0,0,210,4846,18256,190,734,30,67,160,477,210,1437,160,1488,180,514,330,17918,330,2627,0,0,190,380,0,0,50,37,60,66,0,0,180,309,0,0,40,191,330,3127,0,0,0,0,0,0,50,41,330,2247,330,2398,0,0,0,0,70,289,270,1014
90001,160,30,100,40,130,510,250,24676,160,25017,150,17851,70,35,20,71,0,0,70,140,0,0,20,88,0,0,20,477,0,0,0,0,0,20,2967,40,186,0,0,0,0,0,0,0,0,0,0,0,0,0,0,130,4063,20530,120,1362,0,0,100,447,130,1980,90,1086,100,429,160,18375,160,3889,30,124,80,145,0,0,0,0,0,0,0,0,60,74,0,0,0,0,160,3868,0,0,0,0,0,0,0,0,160,3723,160,3897,0,0,0,0,60,702,100,474
90001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
90002,12150,6330,1460,4330,9580,27240,14070,167261,12150,170095,9440,126482,610,73,50,19,40,9,150,49,3610,34019,50,9,50,196,380,3133,0,900,4222,70,84,30,116,3630,2834,30,7,0,0,0,0,0,0,170,133,50,103,0,0,540,7237,8945,170,124,300,175,360,987,530,1621,390,3444,260,367,3750,18438,3750,1927,0,0,1800,618,0,0,100,35,540,242,240,39,980,300,0,0,3400,4983,11540,39973,6940,23487,6550,19268,6380,8439,1520,1330,2210,1310,5520,6298,0,0,0,0,930,642,10960,34691
90002,5030,1510,1490,1980,4120,14410,7890,173280,5030,174335,4760,159099,790,52,60,41,40,17,620,351,560,6015,70,-10,40,237,320,5075,0,390,2070,180,831,0,0,710,1055,50,11,0,0,0,0,40,107,190,179,50,99,0,0,1330,21281,49334,840,1008,440,489,900,2430,1320,4684,920,7738,960,2253,4550,63887,4540,7503,0,0,2840,2550,0,0,330,234,590,523,500,89,2050,1688,30,9,420,1003,4960,20637,2020,4371,1750,3736,2140,3459,560,455,2740,4953,3070,5998,0,0,0,0,420,782,4590,15527
90002,1320,300,600,400,1060,4090,2180,78559,1320,78871,1270,72098,390,

Some observations:

1. The table contains several numerical columns and a column for the ZIP code.
2. For each ZIP code, there are 6 rows.  Each row for a ZIP code has data from tax returns in one *income bracket* -- a group of people who make between some income and some other income.  
3. According to the IRS documentation, all the numerical columns are *totals* -- either total numbers of returns that fall into various categories, or total amounts of money (in thousands of dollars) from returns in those categories.  For example, the column `'N02650'` is the number of returns that included a total income amount, and `'A02650'` is the total amount of total income (in thousands of dollars) from those returns.

**Question 1.0.** Since we don't care about income brackets, but we do care about totals per ZIP code, let's group together our income data by ZIP code. Assign the name `income_by_zipcode` to a table with just one row per ZIP code. When you group according to ZIP code, the remaining columns should be summed. In other words, for any other column such as `'N02650'`, the value of `'N02650'` in a row corresponding to ZIP code 90210 (for example) should be the sum of the values of `'N02650'` in the 6 rows of `income_raw` corresponding to ZIP code 90210.

In [7]:
income_by_zipcode = income_raw.group('ZIP', sum)
income_by_zipcode

ZIP,N1 sum,MARS1 sum,MARS2 sum,MARS4 sum,PREP sum,N2 sum,NUMDEP sum,A00100 sum,N02650 sum,A02650 sum,N00200 sum,A00200 sum,N00300 sum,A00300 sum,N00600 sum,A00600 sum,N00650 sum,A00650 sum,N00700 sum,A00700 sum,N00900 sum,A00900 sum,N01000 sum,A01000 sum,N01400 sum,A01400 sum,N01700 sum,A01700 sum,SCHF sum,N02300 sum,A02300 sum,N02500 sum,A02500 sum,N26270 sum,A26270 sum,N02900 sum,A02900 sum,N03220 sum,A03220 sum,N03300 sum,A03300 sum,N03270 sum,A03270 sum,N03150 sum,A03150 sum,N03210 sum,A03210 sum,N03230 sum,A03230 sum,N03240 sum,A03240 sum,N04470 sum,A04470 sum,A00101 sum,N18425 sum,A18425 sum,N18450 sum,A18450 sum,N18500 sum,A18500 sum,N18300 sum,A18300 sum,N19300 sum,A19300 sum,N19700 sum,A19700 sum,N04800 sum,A04800 sum,N05800 sum,A05800 sum,N09600 sum,A09600 sum,N07100 sum,A07100 sum,N07300 sum,A07300 sum,N07180 sum,A07180 sum,N07230 sum,A07230 sum,N07240 sum,A07240 sum,N07220 sum,A07220 sum,N07260 sum,A07260 sum,N09400 sum,A09400 sum,N10600 sum,A10600 sum,N59660 sum,A59660 sum,N59720 sum,A59720 sum,N11070 sum,A11070 sum,N10960 sum,A10960 sum,N06500 sum,A06500 sum,N10300 sum,A10300 sum,N85330 sum,A85330 sum,N85300 sum,A85300 sum,N11901 sum,A11901 sum,N11902 sum,A11902 sum
90001,20970,9010,4930,6980,17360,53540,28170,527457,20970,531772,17740,456544,2070,241,170,208,130,68,1530,1258,4220,40103,190,183,150,1531,890,11571,0,1600,8137,380,2338,90,3903,4520,4161,120,32,0,0,0,0,50,152,470,370,110,179,0,0,2790,49827,136004,1820,4543,850,1020,1910,5712,2770,13037,1900,17162,1950,4222,11460,180741,11440,24020,30,124,6350,5583,0,0,610,404,1380,1030,950,164,4530,3897,40,13,3820,6064,19960,77235,8480,25093,7770,20943,9210,13221,2050,1594,7640,18417,11060,24663,0,0,0,0,2110,3164,18460,56128
90002,18960,8230,3830,6800,15120,47200,24850,462823,18960,467128,15910,396088,1960,227,170,72,130,30,1450,1196,4310,40865,170,257,130,985,950,14167,0,1450,7131,360,2344,30,116,4610,4305,130,29,0,0,0,0,60,161,470,411,100,202,0,0,2910,50033,133338,1880,3809,900,1003,2070,5698,2890,12370,2100,18397,2060,5197,10070,153993,10060,19696,20,24,5660,4942,0,0,610,403,1350,1006,810,138,3910,3362,50,15,3900,6159,18270,73561,8960,27858,8300,23004,8710,12170,2280,1943,6520,14756,10180,21012,0,0,0,0,1630,2176,17040,55190
90003,26180,11310,5130,9640,20570,64470,33760,612733,26180,618848,21680,518846,2410,393,220,100,170,39,1690,1339,6240,59400,230,275,170,1528,1120,16814,0,1880,9002,490,3204,90,1173,6490,6102,150,39,0,0,30,97,90,262,560,477,160,284,0,0,3300,57436,147488,2110,4120,1000,1089,2250,6165,3280,13216,2290,21225,2260,5650,13410,196510,13410,24756,20,19,7510,6410,0,0,780,497,1780,1335,1060,184,5210,4347,50,13,5620,8885,25030,95254,11910,36094,10940,29504,11950,16777,3190,2789,8680,18348,13880,27416,0,0,0,0,2670,3534,23100,72074
90004,27360,15330,7000,4670,20260,51180,17800,1.61777e+06,27360,1.64943e+06,22010,1.02928e+06,5270,20986,2890,37375,2670,26861,2650,7675,7390,102468,2820,83652,910,18993,1660,36911,0,1840,11778,1310,18705,1480,216013,9000,31663,280,69,240,8545,810,5683,440,2188,1740,1811,260,562,30,555,5510,240787,1.07313e+06,4470,98390,860,996,2910,25109,5470,126876,2560,45167,4390,37387,18590,1.07024e+06,18510,259534,1010,10245,6420,7893,990,1973,620,361,1680,1378,990,175,3010,2817,50,25,6600,17090,25170,318535,7180,14936,5470,11397,4720,6107,1990,1554,16120,252178,20150,276712,690,2166,820,4768,6360,32663,19660,59388
90005,15430,8550,3870,2830,11210,29910,11130,707020,15430,717290,12610,454410,2230,5575,1000,5358,910,3785,990,2305,3950,46369,1080,19080,270,5008,590,11898,0,830,5514,450,5601,650,112194,4900,10270,120,29,50,1631,250,1436,220,1064,990,990,150,367,0,0,2170,91544,387528,1620,37572,460,514,1060,8951,2170,47820,980,16705,1580,10994,9720,433364,9700,99692,260,2993,3240,3165,260,264,300,193,920,752,470,87,1700,1480,0,0,3550,7585,14110,125690,4140,8192,3160,6289,3220,4378,1110,881,8210,96559,10540,105653,180,545,190,713,3550,14752,11260,28426
90006,22630,11470,5400,5630,17840,47590,20210,563530,22630,571157,18360,466220,2130,841,560,949

In [8]:
_ = project1.grade('q10')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



**Question 1.1.** Relabel the columns in `income_by_zipcode` to match the labels in `income_raw`; you probably modified all the names slightly in the previous question. 

*Hint:* Inspect `income_raw.labels` and `income_by_zipcode.labels` to find the differences you need to change.

*Hint 2:* Since there are many columns, it will be easier to relabel each of them by using a `for` statement. See [here](https://www.inferentialthinking.com/chapters/08/2/iteration.html)  in the textbook for details.

*Hint 3:* You can use the `replace` method of a string to remove excess content. See [lab02](https://jupyterhub.ucsd.edu/user-redirect/interact?account=ucsd-ets&repo=dsc10-fa17&branch=master&path=labs/lab02/lab02.ipynb) for examples.

*Hint 4:* To create a new table from an existing table with one label replaced, use `relabeled`. To **change** a label in an existing table permanently, use `relabel`. Both methods take two arguments: the old label and the new label. You can solve this problem with either one, but `relabel` is simpler.

In [9]:
#first step: figure out how the replace string method works
#second step: extract one of the strings that needs fixing
#third step: fix it and relabel the column (say, column 3)
#fourth step: now that we know how to relabel a column, do it for all the columns (iterate)

bad_labels = income_by_zipcode.labels

for i in np.arange(1, income_by_zipcode.num_columns):
    income_by_zipcode.relabel(bad_labels[i], bad_labels[i].replace(" sum", ""))
income_by_zipcode

ZIP,N1,MARS1,MARS2,MARS4,PREP,N2,NUMDEP,A00100,N02650,A02650,N00200,A00200,N00300,A00300,N00600,A00600,N00650,A00650,N00700,A00700,N00900,A00900,N01000,A01000,N01400,A01400,N01700,A01700,SCHF,N02300,A02300,N02500,A02500,N26270,A26270,N02900,A02900,N03220,A03220,N03300,A03300,N03270,A03270,N03150,A03150,N03210,A03210,N03230,A03230,N03240,A03240,N04470,A04470,A00101,N18425,A18425,N18450,A18450,N18500,A18500,N18300,A18300,N19300,A19300,N19700,A19700,N04800,A04800,N05800,A05800,N09600,A09600,N07100,A07100,N07300,A07300,N07180,A07180,N07230,A07230,N07240,A07240,N07220,A07220,N07260,A07260,N09400,A09400,N10600,A10600,N59660,A59660,N59720,A59720,N11070,A11070,N10960,A10960,N06500,A06500,N10300,A10300,N85330,A85330,N85300,A85300,N11901,A11901,N11902,A11902
90001,20970,9010,4930,6980,17360,53540,28170,527457,20970,531772,17740,456544,2070,241,170,208,130,68,1530,1258,4220,40103,190,183,150,1531,890,11571,0,1600,8137,380,2338,90,3903,4520,4161,120,32,0,0,0,0,50,152,470,370,110,179,0,0,2790,49827,136004,1820,4543,850,1020,1910,5712,2770,13037,1900,17162,1950,4222,11460,180741,11440,24020,30,124,6350,5583,0,0,610,404,1380,1030,950,164,4530,3897,40,13,3820,6064,19960,77235,8480,25093,7770,20943,9210,13221,2050,1594,7640,18417,11060,24663,0,0,0,0,2110,3164,18460,56128
90002,18960,8230,3830,6800,15120,47200,24850,462823,18960,467128,15910,396088,1960,227,170,72,130,30,1450,1196,4310,40865,170,257,130,985,950,14167,0,1450,7131,360,2344,30,116,4610,4305,130,29,0,0,0,0,60,161,470,411,100,202,0,0,2910,50033,133338,1880,3809,900,1003,2070,5698,2890,12370,2100,18397,2060,5197,10070,153993,10060,19696,20,24,5660,4942,0,0,610,403,1350,1006,810,138,3910,3362,50,15,3900,6159,18270,73561,8960,27858,8300,23004,8710,12170,2280,1943,6520,14756,10180,21012,0,0,0,0,1630,2176,17040,55190
90003,26180,11310,5130,9640,20570,64470,33760,612733,26180,618848,21680,518846,2410,393,220,100,170,39,1690,1339,6240,59400,230,275,170,1528,1120,16814,0,1880,9002,490,3204,90,1173,6490,6102,150,39,0,0,30,97,90,262,560,477,160,284,0,0,3300,57436,147488,2110,4120,1000,1089,2250,6165,3280,13216,2290,21225,2260,5650,13410,196510,13410,24756,20,19,7510,6410,0,0,780,497,1780,1335,1060,184,5210,4347,50,13,5620,8885,25030,95254,11910,36094,10940,29504,11950,16777,3190,2789,8680,18348,13880,27416,0,0,0,0,2670,3534,23100,72074
90004,27360,15330,7000,4670,20260,51180,17800,1.61777e+06,27360,1.64943e+06,22010,1.02928e+06,5270,20986,2890,37375,2670,26861,2650,7675,7390,102468,2820,83652,910,18993,1660,36911,0,1840,11778,1310,18705,1480,216013,9000,31663,280,69,240,8545,810,5683,440,2188,1740,1811,260,562,30,555,5510,240787,1.07313e+06,4470,98390,860,996,2910,25109,5470,126876,2560,45167,4390,37387,18590,1.07024e+06,18510,259534,1010,10245,6420,7893,990,1973,620,361,1680,1378,990,175,3010,2817,50,25,6600,17090,25170,318535,7180,14936,5470,11397,4720,6107,1990,1554,16120,252178,20150,276712,690,2166,820,4768,6360,32663,19660,59388
90005,15430,8550,3870,2830,11210,29910,11130,707020,15430,717290,12610,454410,2230,5575,1000,5358,910,3785,990,2305,3950,46369,1080,19080,270,5008,590,11898,0,830,5514,450,5601,650,112194,4900,10270,120,29,50,1631,250,1436,220,1064,990,990,150,367,0,0,2170,91544,387528,1620,37572,460,514,1060,8951,2170,47820,980,16705,1580,10994,9720,433364,9700,99692,260,2993,3240,3165,260,264,300,193,920,752,470,87,1700,1480,0,0,3550,7585,14110,125690,4140,8192,3160,6289,3220,4378,1110,881,8210,96559,10540,105653,180,545,190,713,3550,14752,11260,28426
90006,22630,11470,5400,5630,17840,47590,20210,563530,22630,571157,18360,466220,2130,841,560,949,490,631,980,1084,5730,54327,610,3191,230,1827,650,7700,0,1220,6570,470,3388,340,11002,6500,7628,120,28,0,0,130,402,190,898,920,910,130,305,0,0,2040,41866,130856,1380,5958,550,636,970,3901,2020,11393,940,11218,1400,4235,12680,233059,12640,33292,70,260,4980,4065,100,12,390,247,1310,1031,760,147,3080,2517,0,0,5320,8272,20900,72585,7800,18122,6230,14213,6600,8904,1780,1391,9890,29220,14110,37674,0,0,20,82,4350,6859,17660,41538
90007,11710,

In [10]:
_ = project1.grade('q11')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



**Question 1.2.**
Create a table called `income` with  one row per ZIP code and the following columns.

1. A `ZIP` column with the same contents as `'ZIP'` from `income_by_zipcode`.
2. A `returns` column containing the total number of tax returns that include a total income amount (column `'N02650'` from `income_by_zipcode`).
3. A `total` column containing the total income in all tax returns in thousands of dollars (column `'A02650'` from `income_by_zipcode`).
4. A `farmers` column containing the number of farmer returns (column `'SCHF'` from `income_by_zipcode`).

In [11]:
income = Table().with_columns([
        'ZIP', income_by_zipcode.column('ZIP'),
        'returns', income_by_zipcode.column('N02650'),
        'total', income_by_zipcode.column('A02650'),
        'farmers', income_by_zipcode.column('SCHF')
        ])
income.set_format('total', NumberFormatter(0)).sort("farmers", descending=True)

ZIP,returns,total,farmers
99999,397330,"29,613,733",3240
92028,20360,"1,323,779",820
95361,13500,"875,411",570
92082,8640,"628,722",480
95963,6100,"263,960",470
95358,11880,"520,444",400
95320,5370,"359,817",380
93292,15050,"756,000",380
94558,31930,"2,609,616",360
93657,13980,"619,423",360


In [12]:
_ = project1.grade('q12')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



**Question 1.3.** What is the average total income reported on all California tax returns that include a total income amount? **Express the answer in *dollars* as an `int` rounded to the nearest dollar.**

In [13]:
#all the income out of all the returns
#remember 'total' is measured in thousands of dollars

total_income_dollars = income.column('total').sum()*1000
total_returns = income.column('returns').sum()
avg_total_income = total_income_dollars/total_returns
avg_total_income = round(avg_total_income)
avg_total_income

72832.0

**Question 1.4.** All ZIP codes with less than 100 returns (or some other special conditions) are grouped together into one ZIP code with a special code. Remove the row for that ZIP code from the `income` table. *Hint:* This ZIP code value has far more returns than any of the other ZIP codes.

*Hint:* To **remove** a row in the `income` table using `where`, assign `income` to the smaller table using the following expression structure:

    income = income.where(...)
    
*Hint 2:* Each ZIP code is represented as a string, not an `int`.

In [14]:
income.sort('returns', descending=True)
income = income.where('ZIP', are.not_equal_to('99999'))
income.sort('farmers', descending=True)

ZIP,returns,total,farmers
92028,20360,"1,323,779",820
95361,13500,"875,411",570
92082,8640,"628,722",480
95963,6100,"263,960",470
95358,11880,"520,444",400
95320,5370,"359,817",380
93292,15050,"756,000",380
94558,31930,"2,609,616",360
93657,13980,"619,423",360
95380,15460,"638,187",340


In [15]:
_ = project1.grade('q14')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



**Question 1.5**. Among the tax returns in California for ZIP codes represented in the `incomes` table, is there an association between income and living in a ZIP code with a higher-than-average proportion of farmers?

Answer the question by comparing the average incomes for two groups of *tax returns*: those in ZIP codes with a greater-than-average proportion of farmers and those in ZIP codes with a less-than-average (or average) proportion. Make sure both of these values are displayed (preferably in a table). *Then, describe your findings.*

##### Please disregrad the map drawing part of the code below. You can comment it out for submission and it will not be graded


In [16]:
# Build and display a table with two rows:
#   1) incomes of returns in ZIP codes with a greater-than-average proportion of farmers
#   2) incomes of returns in other ZIP codes
farm_proportion = income.column('farmers')/income.column('returns')
avg_prop_farmers = np.mean(farm_proportion)
income_by_farm_proportion = income.with_column('Farm Proportion', farm_proportion)
income_by_farm_proportion.sort('farmers', descending=True)



ZIP,returns,total,farmers,Farm Proportion
92028,20360,"1,323,779",820,0.040275
95361,13500,"875,411",570,0.0422222
92082,8640,"628,722",480,0.0555556
95963,6100,"263,960",470,0.0770492
95358,11880,"520,444",400,0.03367
95320,5370,"359,817",380,0.0707635
93292,15050,"756,000",380,0.0252492
94558,31930,"2,609,616",360,0.0112747
93657,13980,"619,423",360,0.0257511
95380,15460,"638,187",340,0.0219922


In [17]:
high_farmers = income_by_farm_proportion.where('Farm Proportion', are.above(avg_prop_farmers))
high_farmers

ZIP,returns,total,farmers,Farm Proportion
91906,1440,"65,980",30,0.0208333
91935,4220,"383,661",50,0.0118483
92003,2570,"214,821",250,0.0972763
92028,20360,"1,323,779",820,0.040275
92036,1530,"92,064",30,0.0196078
92061,1390,"112,032",50,0.0359712
92065,16230,"1,140,168",220,0.0135551
92070,540,"26,955",20,0.037037
92082,8640,"628,722",480,0.0555556
92086,550,"23,629",20,0.0363636


In [18]:
low_farmers = income_by_farm_proportion.where('Farm Proportion', are.not_above(avg_prop_farmers))
low_farmers

ZIP,returns,total,farmers,Farm Proportion
90001,20970,"531,772",0,0
90002,18960,"467,128",0,0
90003,26180,"618,848",0,0
90004,27360,"1,649,431",0,0
90005,15430,"717,290",0,0
90006,22630,"571,157",0,0
90007,11710,"315,581",0,0
90008,14710,"668,523",0,0
90010,2210,"320,471",0,0
90011,36670,"864,961",0,0


In [19]:
#all the income out of all the returns
#remember 'total' is measured in thousands of dollars

high_farmers_total_income_dollars = high_farmers.column('total').sum()*1000
high_farmers_total_returns = high_farmers.column('returns').sum()
high_farmers_avg_total_income = high_farmers_total_income_dollars/high_farmers_total_returns
high_farmers_avg_total_income = round(high_farmers_avg_total_income)
high_farmers_avg_total_income

60428.0

In [20]:
#all the income out of all the returns
#remember 'total' is measured in thousands of dollars

low_farmers_total_income_dollars = low_farmers.column('total').sum()*1000
low_farmers_total_returns = low_farmers.column('returns').sum()
low_farmers_avg_total_income = low_farmers_total_income_dollars/low_farmers_total_returns
low_farmers_avg_total_income = round(low_farmers_avg_total_income)
low_farmers_avg_total_income

73917.0

######  Solution:
It is true that there is an association between income and living in a ZIP code with a higher proportion of farmers. This is a negative association, meaning ZIP codes with a higher proportion of farmers have lower average incomes, whereas zip codes with a lower proportion of farmers have higher average incomes. One reason for this is that farmers don't make a lot of money. Another reason is that high-earning people tend to work in cities, and cities do not have a lot of farmland.

ZIP codes cover all the land in California and do not overlap. Here's a map of all of them.

![CA ZIP Codes](california-zip-code-map.jpg)

**Question 1.7.** Among the ZIP codes represented in the `incomes` table, is there an association between high average income and some aspect of the ZIP code's location? If so, describe one aspect of the location that is clearly associated with high income.

Answer the question by drawing a map of all ZIP codes that have an average income above 100,000 dollars. *Then, describe an association that you observe.*

In order to create a map of certain ZIP codes, you need to 
- Construct a table containing only the ZIP codes of interest, called `high_average_zips`,
- Join `high_average_zips` with the `zip_features` table to find the region for each ZIP code of interest,
- Call `Map(...)` on the column of features (provided).

In [21]:
income.with_column('Average Income', income.column('total')/income.column('returns'))

ZIP,returns,total,farmers,Average Income
90001,20970,"531,772",0,25.3587
90002,18960,"467,128",0,24.6376
90003,26180,"618,848",0,23.6382
90004,27360,"1,649,431",0,60.2862
90005,15430,"717,290",0,46.4867
90006,22630,"571,157",0,25.2389
90007,11710,"315,581",0,26.9497
90008,14710,"668,523",0,45.4468
90010,2210,"320,471",0,145.01
90011,36670,"864,961",0,23.5877


In [22]:
# Write code to draw a map of only the high-income ZIP codes
zip_features = Table.from_records(zips.features)

high_average_zips = income.with_column('Average Income', 
                                       income.column('total')/income.column('returns')).where('Average Income', are.above(100))

high_zips_with_region = high_average_zips.join('ZIP', zip_features)

Map(list(high_zips_with_region.column('feature')), width=400, height=300)

######  Solution:
Proximity to the coast is clearly associated with higher income. This makes sense, because real estate is much more expensive near the ocean, as this is a desirable place to live.

In [23]:
_ = project1.grade('q17')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



Part 2: Water Usage
======
We will now investigate water usage in California. The `usage` table contains three columns:

- `PWSID`: The Public Water Supply Identifier of the district
- `Population`: Estimate of average population served in 2015
- `Water`: Average residential water use (gallons per person per day) in 2014-2015

In [49]:
# Run this cell to create the usage table

usage_raw.set_format(4, NumberFormatter)
max_pop = usage_raw.select([0, 'population']).group(0, max).relabeled(1, 'Population')
avg_water = usage_raw.select([0, 'res_gpcd']).group(0, np.mean).relabeled(1, 'Water')
usage = max_pop.join('pwsid', avg_water).relabeled(0, 'PWSID')
usage


PWSID,Population,Water
1010001,13960,209.909
1010003,108227,156.818
1010007,520159,107.273
1010018,14339,155.8
1010019,11685,225.818
1010024,24637,134.909
1010026,8495,169.571
1010027,24194,82.7273
1010029,25664,116.636
110001,340000,70.7


###### To view a map of the water districts, please click [here](https://glowingshadow.ml:8888/notebooks/dsc10-fa17/projects/proj1/project1.ipynb) (Password is DSC10). Please be patient, this takes some time to load.

**Question 2.2.** Based on the map above, which part of California appears to use more water per person, the San Francisco area or the Los Angeles area? 

######  Solution:
The Los Angeles area uses more water per person. The colors on the map represent per capita water usage, with darker colors representing more water usage. The LA area appears much darker on the map than the San Francisco area.

Next, we will try to match each ZIP code with a water district. ZIP code boundaries do not always line up with water districts, and one water district often covers multiple ZIP codes, so this process is imprecise. It is even the case that some water districts overlap each other. Nonetheless, we can continue our analysis by matching each ZIP code to the water district with the largest geographic overlap.

The table `wd_vs_zip` describes the proportion of land in each ZIP code that is contained in each water district and vis versa. (The proportions are approximate because they do not correctly account for discontiguous districts, but they're mostly accurate.)

In [50]:
wd_vs_zip.show(5)

PWSID,ZIP,District in ZIP,ZIP in District
0110001,94536,9.41%,68.51%
0110001,94538,18.87%,67.31%
0110001,94539,13.13%,44.36%
0110005,94541,1.61%,68.11%
0110006,94541,18.68%,98.46%


**Question 2.3.** Complete the `district_for_zip` function that takes a ZIP code. It returns the PWSID with the largest value of `ZIP in District` for that `zip_code`, if that value is at least 50%. Otherwise, it returns the string `'No District'`.

In [51]:
def district_for_zip(zip_code):
    zip_code = str(zip_code) # Ensure that the ZIP code is a string, not an integer
    districts = wd_vs_zip.where('ZIP', zip_code).sort('ZIP in District', descending=True)
    if districts.num_rows > 0:
        max_percent = districts.column('ZIP in District').item(0)
    else:
        return 'No District'
    if max_percent >= 0.5:
        return districts.column('PWSID').item(0)
    else:
        return 'No District'

district_for_zip(94709)

'0110005'

In [52]:
_ = project1.grade('q23')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



This function can be used to associate each ZIP code in the `income` table with a `PWSID` and discard ZIP codes that do not lie (mostly) in a water district.

In [53]:
zip_pwsids = income.apply(district_for_zip, 'ZIP')
income_with_pwsid = income.with_column('PWSID', zip_pwsids).where(zip_pwsids != "No District")
income_with_pwsid.set_format(2, NumberFormatter(0)).show(5)

ZIP,returns,total,farmers,PWSID
90001,20970,"531,772",0,1910067
90022,26680,"767,484",0,1910036
90024,14690,"4,395,487",20,1910067
90025,25110,"4,019,082",20,1910067
90034,29950,"1,828,572",0,1910067


**Question 2.4.** Create a table called `district_data` with one row per PWSID and the following columns:

- `PWSID`: The ID of the district
- `Population`: Population estimate
- `Water`: Annual water usage (per capita)
- `Income`: Average income in dollars of all tax returns in ZIP codes that are (mostly) contained in the district according to `income_with_pwsid`.

*Hint*: First create a `district_income` table that sums the incomes and returns for ZIP codes in each water district.

In [68]:
district_income = income_with_pwsid.group('PWSID', sum)
district_income

PWSID,ZIP sum,returns sum,total sum,farmers sum
0110001,,128720,1.0173e+07,30
0110005,,512190,4.22542e+07,140
0110006,,83840,4.43711e+06,0
0110008,,20000,3.26515e+06,50
0110009,,39450,5.28242e+06,0
0410002,,14340,722747,70
0410006,,1120,43367,60
0410011,,5070,226663,0
0710001,,46000,2.46334e+06,0
0710003,,39320,2.90629e+06,50


In [69]:
usage

PWSID,Population,Water
1010001,13960,209.909
1010003,108227,156.818
1010007,520159,107.273
1010018,14339,155.8
1010019,11685,225.818
1010024,24637,134.909
1010026,8495,169.571
1010027,24194,82.7273
1010029,25664,116.636
0110001,340000,70.7


In [70]:
#This part is optional. If you use it, you must use the later part where you pad with 0's.

#def pad_with_0(x):
#    if len(x)==6:
#        return "0"+x
#    else:
#        return x
    
#usage = usage.with_column( "PWSID", usage.apply(pad_with_0, "PWSID"))
#usage

In [71]:
district_data = district_income.join('PWSID', usage)
district_data = district_data.with_column('Income', district_data.column('total sum')/district_data.column('returns sum')*1000)
district_data = district_data.drop('ZIP sum', 'returns sum', 'total sum', 'farmers sum')

#This part is optional.
#water_column = district_data.column('Water')
#district_data = district_data.with_column('Water', water_column*365)
                                                         

district_data.set_format(['Population', 'Water', 'Income'], NumberFormatter(0))
district_data 

PWSID,Population,Water,Income
0110001,"340,000",71,"79,032"
0110005,"1,390,000",76,"82,497"
0110006,"151,037",57,"52,924"
0110008,"73,067",97,"163,257"
0110009,"79,547",69,"133,902"
0410002,"101,447",142,"50,401"
0410006,"18,300",286,"38,721"
0410011,"9,615",92,"44,707"
0710001,"106,455",110,"53,551"
0710003,"197,536",102,"73,914"


In [72]:
_ = project1.grade('q24')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



**Question 2.5.** The `bay_districts` table gives the names of all water districts in the San Francisco Bay Area. Is there an association between water usage and income among Bay Area water districts? Use the tables you have created to compare water usage between the 10 Bay Area water districts with the highest average income and the rest, then describe the association. *Do not include any districts in your analysis for which you do not have income information.*

The names below are just suggestions; you may perform the analysis in any way you wish.

In [73]:
bay_districts = Table.read_table('bay_districts.csv')

def convert_to_string(x):
    return str(x)

pwsid_string_array = bay_districts.apply(convert_to_string, 'pwsid')

bay_districts = bay_districts.with_column('PWSID', pwsid_string_array).drop('pwsid')

#This part is optional, depending on whether you padded with 0s previously.
#bay_districts = bay_districts.with_column( "PWSID", bay_districts.apply(pad_with_0, "PWSID"))

bay_districts
district_data

PWSID,Population,Water,Income
0110001,"340,000",71,"79,032"
0110005,"1,390,000",76,"82,497"
0110006,"151,037",57,"52,924"
0110008,"73,067",97,"163,257"
0110009,"79,547",69,"133,902"
0410002,"101,447",142,"50,401"
0410006,"18,300",286,"38,721"
0410011,"9,615",92,"44,707"
0710001,"106,455",110,"53,551"
0710003,"197,536",102,"73,914"


In [74]:
bay_water_vs_income = bay_districts.join('PWSID', district_data)
bay_water_vs_income.sort('Income', descending=True)

PWSID,District,Population,Water,Income
4110006,California Water Service Company Bear Gulch,"58,895",170,"1,160,494"
4310001,California Water Service Company Los Altos/Suburban,"68,163",125,"349,183"
4310009,"Palo Alto, City of","64,403",89,"334,057"
4110017,"Menlo Park, City of","16,066",75,"278,733"
2110002,Marin Municipal Water District,"188,200",86,"176,643"
4310007,"Mountain View, City of","76,781",66,"138,570"
4110021,Estero Municipal Improvement District,"37,165",66,"137,893"
4110001,Mid-Peninsula Water District,"26,730",84,"137,537"
4110008,California Water Service Company Mid Peninsula,"135,918",70,"135,967"
3810011,San Francisco Public Utilities Commission,"846,601",44,"118,253"


In [75]:
top_10 = bay_water_vs_income.sort('Income', descending=True).take(np.arange(10))
top_10

PWSID,District,Population,Water,Income
4110006,California Water Service Company Bear Gulch,"58,895",170,"1,160,494"
4310001,California Water Service Company Los Altos/Suburban,"68,163",125,"349,183"
4310009,"Palo Alto, City of","64,403",89,"334,057"
4110017,"Menlo Park, City of","16,066",75,"278,733"
2110002,Marin Municipal Water District,"188,200",86,"176,643"
4310007,"Mountain View, City of","76,781",66,"138,570"
4110021,Estero Municipal Improvement District,"37,165",66,"137,893"
4110001,Mid-Peninsula Water District,"26,730",84,"137,537"
4110008,California Water Service Company Mid Peninsula,"135,918",70,"135,967"
3810011,San Francisco Public Utilities Commission,"846,601",44,"118,253"


In [76]:
not_top_10 = bay_water_vs_income.sort('Income', descending=True).take(np.arange(10, bay_water_vs_income.num_rows, 1))
not_top_10

PWSID,District,Population,Water,Income
4310014,"Sunnyvale, City of","147,055",66,"118,024"
4110022,"Redwood City, City of","87,696",59,"102,883"
4310012,"Santa Clara, City of","118,459",74,"101,256"
4310011,San Jose Water Company,"990,000",80,"89,001"
4310022,Great Oaks Water Company Incorporated,"99,199",73,"86,801"
4110025,North Coast County Water District,"39,000",53,"85,934"
4110023,"San Bruno, City of","43,798",47,"73,371"
4110009,California Water Service Company South San Francisco,"60,935",45,"70,977"
2810005,"American Canyon, City of","20,080",82,"64,635"
2810003,"Napa, City of","85,679",84,"63,026"


In [77]:
#This part could have been done in either of the two ways below.

#top_10_mean = np.mean(top_10.column('Water'))
#not_top_10_mean = np.mean(not_top_10.column('Water'))

top_10_mean = np.sum(top_10.column('Population')*top_10.column('Water'))/np.sum(top_10.column('Population'))
not_top_10_mean = np.sum(not_top_10.column('Population')*not_top_10.column('Water'))/np.sum(not_top_10.column('Population'))

top_10_mean - not_top_10_mean

-8.9980772315717132

*Complete this one-sentence conclusion:* 
##### Solution:
In the Bay Area, people in the top 10 highest-income water districts used an average of *approximately 20.2* more gallons of water per person per day than people in the rest of the districts.

**Question 2.6.** In one paragraph, summarize what you have discovered through the analyses in this project and suggest what analysis should be conducted next to better understand California water usage, income, and geography. What additional data would be helpful in performing this next analysis?

##### Conclusion:

In this project, we discovered that areas with more farmers have a lower average incomes than areas with fewer farmers. We also saw that richer people tend to live near the coast. We saw that LA area residents use a lot of water. Finally, we saw that in the Bay Area, zip codes with higher incomes used more water per capita than zip codes with lower incomes. To continue this analysis, we might want to explore if the trend we observed in the Bay Area is true throughout California. In particular, is high income associated with high water usage throughout California? We saw this was true in the Bay Area, but we also saw that farmers do not have high incomes, yet one would expect that in areas with lots of farmers, the per capita water use is high, since farming uses a lot of water. It might also be informative to compare how water is used in different parts of the state, and to do this, we would need to know the distribution of water usage within each water district or within each zip code. That is, what percentage of water is used for agriculture, landscaping, drinking, home use (showers, sinks, toilets), etc.

Congratulations - you've finished Project 1 of DSC 10! 

To submit:

1. Select `Run All` from the `Cell` menu to ensure that you have executed all cells, including the test cells.  Make sure that the visualizations you create are actually displayed.
3. Read through the notebook to make sure everything is fine.
4. Submit using the cell below.

If you have problems submitting your project come to office hours for assistance. The office hours
schedule appears on [here](https://sites.google.com/a/eng.ucsd.edu/dsc10-fall-2017/staff-hours).


In [78]:
_ = project1.submit()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'project1.ipynb'.
Submit... 0.0% complete
Could not submit: Late Submission of ucsd/dsc10/fa17/project1
Backup... 100% complete



In [79]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [project1.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------

If you want, draw some more maps below.

In [80]:
# Your extensions here (completely optional)